In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!pip install catboost lightgbm shap -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.4 MB/s eta 0:00:00


Imports and Setup

In [ ]:
# For data handling and display
import pandas as pd
from IPython.display import display

# Machine learning models and ensemble
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import VotingClassifier



Load Data

In [ ]:
# URLs to the raw CSV files stored in the project's GitHub repository
url_train = 'https://raw.githubusercontent.com/Assignment3StarshipTitanic/Assignment_3_StarshipTitanic/main/data/train.csv'
url_test = 'https://raw.githubusercontent.com/Assignment3StarshipTitanic/Assignment_3_StarshipTitanic/main/data/test.csv'

# Load training and test datasets from the URLs into Pandas DataFrames
train_df = pd.read_csv(url_train)
test_df = pd.read_csv(url_test)

# (Optional) Load sample submission file if URL provided
sample_submission_df = pd.read_csv(url_sample)

# Display the first few rows of the training dataset
print("Train Dataset")
display(train_df.head())

# Display the first few rows of the test dataset
print("\nTest Dataset")
display(test_df.head())


Train Dataset


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True



 Test Dataset


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


Model Training with Ensemble Learning (CatBoost + LightGBM Voting Classifier)

In [ ]:
# set up CatBoost and LightGBM models
cat = CatBoostClassifier(verbose=0, random_seed=42)
lgb = LGBMClassifier(random_state=42)

# combine both using soft voting
voting = VotingClassifier(estimators=[
    ('cat', cat),
    ('lgb', lgb)
], voting='soft')

# train individual models
cat.fit(X_train_scaled, y_train)
lgb.fit(X_train_scaled, y_train)

# train the ensemble model
voting.fit(X_train_scaled, y_train)
